In [1]:
%matplotlib inline
import cv2
import numpy as np
import os
import sys
import time
from matplotlib import pyplot as plt
from IPython import display


In [2]:
screen_x_eff, screen_y_eff = 1125,1958
jumper_foot_offset = 20
holdDt = 1.39
tap_x, tap_y = 600,1000

background_xy = (10,980)
header_y,foot_y = 400,1200


In [4]:
#load jumper template
jumper_template = cv2.imread('jumper.png')
template_h,template_w = jumper_template.shape[0:2]

In [5]:
def jump(distance):
    dt = int(holdDt * distance)
    rand_tapxy = np.random.randn(4,)*15 #ad some randomness in the tap location
    cmd_msg = 'adb shell input swipe {x1} {y1} {x2} {y2} {dt}'.format(
        x1=tap_x+int(rand_tapxy[0]),y1=tap_y+int(rand_tapxy[1]),
        x2=tap_x+int(rand_tapxy[2]),y2=tap_y+int(rand_tapxy[3]),
        dt=dt)
    os.system(cmd_msg)
    return dt    

In [6]:
def find_jumper_xy(img):
    #matching jumper
    res = cv2.matchTemplate(img,jumper_template,cv2.TM_SQDIFF_NORMED) #find jumper template matching
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    if min_val>0.3: #fail to find a match
        print(min_val,'no matching!')
        return -1,-1,img
    else:
        print(min_val,'matching!')        
    
    top_left = min_loc
    bottom_right = (top_left[0] + template_w, top_left[1] + template_h)

    jumper_xy = (top_left[0]+int(template_w*0.5), top_left[1]+template_h-jumper_foot_offset) #jumper base location
        
    background_color = img[background_xy[1],background_xy[0],:]

    img[top_left[1]:bottom_right[1]+1,top_left[0]:bottom_right[0]+1,:]=background_color
        
    return jumper_xy,img

In [7]:
def find_target_xy(img,jumper_xy):
    
    im_blur = cv2.GaussianBlur(img,(5,5),0)
    canny = cv2.Canny(im_blur, 1, 10) 
    canny[0:header_y,:] = 0
    canny[foot_y:,:] = 0

    y_top = np.nonzero([max(row) for row in canny])[0][0]
    x_top_left = np.min(np.nonzero(canny[y_top]))
    x_top_right = np.max(np.nonzero(canny[y_top]))
    x_top = int(np.mean(np.nonzero(canny[y_top])))

    topcorner_xy = (x_top,y_top);

    #search left boundary
    target_color = img[y_top + 2,x_top];
    y_current = y_top;
    x_current = x_top;
    while True:
        y_current += 1;
        x_search_range = range(x_current-20,x_current);
        color_diff = np.linalg.norm(img[y_current,x_search_range,:]-target_color,axis=1)
        if np.min(color_diff)<30:
            x_current = x_search_range[np.argmin(color_diff)];
            target_color = img[y_current,x_current];
        else: #found corner
            leftcorner_xy = (x_current,y_current-1);
            break;

    #search right boundary
    target_color = img[y_top+1,x_top];
    y_current = y_top;
    x_current = x_top;
    while True:
        y_current += 1;
        x_search_range = range(x_current+20,x_current,-1);
        color_diff = np.linalg.norm(img[y_current,x_search_range,:]-target_color,axis=1)
        if np.min(color_diff)<30:
            x_current = x_search_range[np.argmin(color_diff)];
            target_color = img[y_current,x_current];
        else: #found corner
            rightcorner_xy = (x_current,y_current-1);
            break;

    img = cv2.line(img,topcorner_xy,leftcorner_xy, (0,255,0), 2)   # highlight where the jumper center is found
    img = cv2.line(img,topcorner_xy,rightcorner_xy, (0,255,0), 2)   # highlight where the jumper center is found
    img = cv2.line(img,leftcorner_xy,rightcorner_xy, (0,255,0), 2)   # highlight where the jumper center is found

    target_xy = ((leftcorner_xy[0]+rightcorner_xy[0])/2,(leftcorner_xy[1]+rightcorner_xy[1])/2);

    return target_xy,img

In [8]:
def find_positions(im_name):
    
    img = cv2.imread(im_name);
    
    jumper_xy,img = find_jumper_xy(img);

    mirrored_xy = (screen_x_eff-jumper_xy[0],screen_y_eff-jumper_xy[1])  #mirror the jumper base location to get the target base location

    target_xy, img = find_target_xy(img,jumper_xy);
    
    img = cv2.circle(img,jumper_xy, 10, 255, 2)               # highlight jumper base location
    img = cv2.circle(img,target_xy, 10, 255, 2)               # highlight target base location
    
    target_xy = ((target_xy[0]+mirrored_xy[0])/2,(target_xy[1]+mirrored_xy[1])/2)
    
    img = cv2.circle(img,target_xy, 10, (0,255,255), 2)               # highlight target base location
    
    distance = np.sqrt(np.square(target_xy[0]-jumper_xy[0])+np.square(target_xy[1]-jumper_xy[1])) #compute jump distance

    
    return jumper_xy,target_xy,distance,img

In [12]:
while True:
    os.system('adb shell screencap /sdcard/1.png');     #take a screenshot
    os.system('adb pull /sdcard/1.png ./scrshot.png');  #download the screenshot to local disk
    
    jumper_xy,target_xy,distance,img = find_positions('scrshot.png');
    
    plt.clf()
    fig=plt.figure(figsize=(18, 16))
    plt.subplot(111)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.xticks([]), plt.yticks([])
    plt.show()

    if distance<0: #fail to find match
        print('failed');
        break;
    jump(distance);
    
    dt = 2.0 + np.random.rand();
    print(dt);
    time.sleep(dt);
    
    
    display.display(plt.gcf())
    display.clear_output(wait=True)   
    

(0.4057019054889679, 'no matching!')


ValueError: too many values to unpack